<a href="https://colab.research.google.com/github/abhishank-gaba/Kaggle-Challenge-Primary-site-classi-cation-in-histopathology-images/blob/master/Google_Collab_Executable_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from keras.models import Sequential 
from keras.layers import Convolution2D as Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from IPython.display import display
from PIL import Image 
from skimage.transform import rescale, resize, downscale_local_mean
from keras.applications.vgg16 import preprocess_input
from keras.regularizers import l2
from google.colab import files






Using TensorFlow backend.


In [0]:
#read data 
train_df = np.load("train_x.npy")

train_label_df = np.load ("train_label.npy")

test_df = np.load ("test_x.npy")


# unique classes
classes = np.unique(train_label_df)
class_dict = {cls: i for i, cls in enumerate(classes)}

# tf readable labels
Y = [np.eye(len(classes))[class_dict[label]] for label in train_label_df]

X_ds = tf.data.Dataset\
    .from_tensor_slices(train_df)

ds_iter = iter(X_ds)

Y_ds = tf.data.Dataset\
    .from_tensor_slices(Y)

ds_iter = iter(Y_ds)

print(next(ds_iter).numpy())

ds = tf.data.Dataset.zip((X_ds, Y_ds))\
    .batch(32)\
    .repeat(1000)

ds_iter = iter(ds)
print(next(ds_iter))


Instructions for updating:
Colocations handled automatically by placer.
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(<tf.Tensor: id=28, shape=(32, 168, 308, 3), dtype=uint8, numpy=
array([[[[156, 191, 223],
         [170, 199, 229],
         [211, 224, 240],
         ...,
         [236, 235, 235],
         [242, 244, 238],
         [250, 252, 249]],

        [[156, 193, 225],
         [163, 197, 231],
         [203, 220, 239],
         ...,
         [242, 241, 237],
         [247, 251, 248],
         [249, 252, 253]],

        [[151, 189, 218],
         [162, 189, 223],
         [205, 219, 238],
         ...,
         [250, 249, 248],
         [248, 253, 252],
         [248, 252, 252]],

        ...,

        [[188, 196, 220],
         [212, 211, 234],
         [218, 220, 234],
         ...,
         [234, 238, 234],
         [244, 247, 246],
         [244, 246, 250]],

        [[155, 185, 214],
         [167, 198, 225],
         [170, 203, 228],
         ...,
       

In [0]:

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005), input_shape=(168, 308, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005), input_shape=(168, 308, 3)),
    keras.layers.MaxPool2D((2, 2)),
        
    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
        
    keras.layers.Conv2D(128, (3, 3),padding='same', activation='relu'),
    keras.layers.Conv2D(128, (3, 3),padding='same', activation='relu'),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Flatten(),

    
    keras.layers.Dense(512, activation='relu'),

    keras.layers.Dense(len(classes))
]);



In [0]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 168, 308, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 168, 308, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 84, 154, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 154, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 84, 154, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 42, 77, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 42, 77, 128)       73856     
__________

In [0]:
optimizer = tf.train.AdamOptimizer(1e-4)
model.compile(optimizer=optimizer, loss=tf.losses.softmax_cross_entropy, metrics=['accuracy'])

model.fit( 
    ds,
    steps_per_epoch=100,
    epochs=10,
    verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 32s 322ms/step - loss: 6.0146 - acc: 0.5275
Epoch 2/10
100/100 [==============================] - 28s 282ms/step - loss: 0.2355 - acc: 0.9441
Epoch 3/10
100/100 [==============================] - 28s 282ms/step - loss: 0.1307 - acc: 0.9801
Epoch 4/10
100/100 [==============================] - 28s 282ms/step - loss: 0.0216 - acc: 1.0000
Epoch 5/10
100/100 [==============================] - 28s 282ms/step - loss: 0.0189 - acc: 1.0000
Epoch 6/10
100/100 [==============================] - 28s 282ms/step - loss: 0.0179 - acc: 1.0000
Epoch 7/10
100/100 [==============================] - 29s 288ms/step - loss: 0.0159 - acc: 1.0000
Epoch 8/10
100/100 [==============================] - 28s 283ms/step - loss: 0.0149 - acc: 1.0000
Epoch 9/10
100/100 [==============================] - 28s 282ms/step - loss: 0.0146 - acc: 1.0000
Epoch 10/10
100/100 [==============================] - 28s 283ms/step 

In [0]:
predictions = model.predict(test_df)
predictions = np.argmax (predictions, axis=1)
print (predictions)

result_df = pd.DataFrame({'Id': list(range(len(predictions))), 'Predicted': predictions})
result_df.to_csv('results.csv', index=False)

files.download('true_results.csv') 

[10  6  8  7 19 11 17 15  1  7  4  6 18  2 11  3  0 12  8 11 12  4 11 16
 15  0 11  5  1  7  7  2 16 13 15 13 15  5  8 19 16  4 11 19 14  2 18  9
  9 11  8 13  1 18  2  5 18 13  0  5 15 12  1  2 17 16  5 10 15  1 18 18
  2  1  1  4 17  8 16 17  3  8  8 10  7 13 14 14 13 10 16 15 15  5 17 17
  6 15 12  3 11  2 14 11  4  9  3 12 17 14  4 15 13 15  8 13  3 15 16 12
  2  3 11 14  6  7  1  1 16 16 11 10  4  0 12 10  3 14 14 18  0  2  9  4
  5  3 10  5 14  0  4 15  5 11 11 15  3  7 18 19  4  0 17  1  2  6  9  8
  4  3 19 13 18 17  3 11  1 17 15  3  9 13 14  8 17 14 16 11  9 10 14  6
 17 19 10  9 11  3 18 11]


MessageError: ignored